In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
#from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from keys import yelp_key


df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)


In [2]:

headers = {'Authorization': 'Bearer %s' % yelp_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city in random_rural_city["NAME"]:
    params = {
        'location': city,
        'limit': 50,
        'offset':1100,
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            rural_dict["Searched"].append(city)
            rural_dict["City"].append(response["businesses"][business]["location"]["city"])
            rural_dict["State"].append(response["businesses"][business]["location"]["state"])
            rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            rural_dict["Restaurant"].append(response["businesses"][business]["name"])
            rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            rural_dict["Searched"].append(city)
            rural_dict["City"].append("No City Found")
            rural_dict["State"].append("No City Found")
            rural_dict["Lat"].append("No City Found")
            rural_dict["Lng"].append("No City Found")
            rural_dict["Restaurant"].append("No City Found")
            rural_dict["Category"].append("No City Found")
            rural_dict["Transactions"].append("No City Found")

KeyError: 'businesses'

In [ ]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()
rural.head()

In [ ]:
headers = {'Authorization': 'Bearer %s' % yelp_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Transactions":[],
    "Total": []
}

url='https://api.yelp.com/v3/businesses/search'
 
for city in random_urban_city["NAME"]:
    params = {
        'limit': 50,
        'offset': 1100,
        'location': city,
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            
            urban_dict["Searched"].append(city)
            urban_dict["Total"].append(response["total"])
            urban_dict["City"].append(response["businesses"][business]["location"]["city"])
            urban_dict["State"].append(response["businesses"][business]["location"]["state"])
            urban_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            urban_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            urban_dict["Restaurant"].append(response["businesses"][business]["name"])
            urban_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            urban_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            urban_dict["Searched"].append(city)
            urban_dict["City"].append("No City Found")
            urban_dict["State"].append("No City Found")
            urban_dict["Lat"].append("No City Found")
            urban_dict["Lng"].append("No City Found")
            urban_dict["Restaurant"].append("No City Found")
            urban_dict["Category"].append("No City Found")
            urban_dict["Transactions"].append("No City Found")
#print(urban_dict["Total"])

In [ ]:
urban = pd.DataFrame.from_dict(urban_dict, orient = "index")
urban = urban.transpose()
urban

In [ ]:
urban["Total"].max()

In [ ]:
urban.to_csv("../urban.csv")
rural.to_csv("../rural.csv")

In [ ]:
urban_vegan = urban.loc[urban["Category"] == "Vegan"]
urban_vegan.head()
rural_vegan = rural.loc[rural["Category"] == "Vegan"]
rural_vegan.head()

In [ ]:
Category.str.findall('Vegan')

In [ ]:
location = list(zip(random_rural_city["NAME"], random_rural_city["STNAME"]))
print(location)
                

In [ ]:
urban_grouped = urban.groupby("Category").max()